In [80]:
import os
workspace = "/workspace"
os.chdir(workspace)

import pickle
import mne
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from tqdm import tqdm
from glob import glob

from mne.time_frequency import tfr_array_morlet
from src import read_config, plot_single_channel, plot_topomap, show_morlet
from src.train import NMSELoss

In [81]:
import torch

pred = torch.rand(128, 1, 64, 1600)
target = torch.ones(128, 1, 64, 1600)

l = NMSELoss()
loss = l(pred, target)

In [82]:
loss

tensor(0.3331)

In [3]:
folder_list = sorted(glob("data/eegmmidb/*"))[5:-7]
len(folder_list)

0

In [4]:
ch_list = list(pd.read_csv("data/eegmmidb/ch_list.csv")["ch_names"])
print(ch_list)

['FC5', 'FC3', 'FC1', 'FCz', 'FC2', 'FC4', 'FC6', 'C5', 'C3', 'C1', 'Cz', 'C2', 'C4', 'C6', 'CP5', 'CP3', 'CP1', 'CPz', 'CP2', 'CP4', 'CP6', 'Fp1', 'Fpz', 'Fp2', 'AF7', 'AF3', 'AFz', 'AF4', 'AF8', 'F7', 'F5', 'F3', 'F1', 'Fz', 'F2', 'F4', 'F6', 'F8', 'FT7', 'FT8', 'T7', 'T8', 'T9', 'T10', 'TP7', 'TP8', 'P7', 'P5', 'P3', 'P1', 'Pz', 'P2', 'P4', 'P6', 'P8', 'PO7', 'PO3', 'POz', 'PO4', 'PO8', 'O1', 'Oz', 'O2', 'Iz']


In [5]:
montage = read_config("data/Montage/64_ch_system.pkl")
new_ch_list = montage.ch_names

In [6]:
new_ch_list

['Fp1',
 'Fpz',
 'Fp2',
 'AF7',
 'AF3',
 'AFz',
 'AF4',
 'AF8',
 'F7',
 'F5',
 'F3',
 'F1',
 'Fz',
 'F2',
 'F4',
 'F6',
 'F8',
 'FT7',
 'FC5',
 'FC3',
 'FC1',
 'FCz',
 'FC2',
 'FC4',
 'FC6',
 'FT8',
 'T9',
 'T7',
 'C5',
 'C3',
 'C1',
 'Cz',
 'C2',
 'C4',
 'C6',
 'T8',
 'T10',
 'TP7',
 'CP5',
 'CP3',
 'CP1',
 'CPz',
 'CP2',
 'CP4',
 'CP6',
 'TP8',
 'P7',
 'P5',
 'P3',
 'P1',
 'Pz',
 'P2',
 'P4',
 'P6',
 'P8',
 'PO7',
 'PO3',
 'POz',
 'PO4',
 'PO8',
 'O1',
 'Oz',
 'O2',
 'Iz']

In [6]:
config = {}

for i, ch in enumerate(new_ch_list):
    if ch in ['Fp1', "Fp2", "Fz", 'T7', "T8", "Pz", "O1", "O2"]:
        config[i] = ch

In [10]:
import json

with open("data/eegmmidb/ch_order.json", "w") as F:
    json.dump(config, F, indent = 4)

In [11]:
dir_list = []
for i in range(1, 110):   
    if i < 10:
        dir_num = f"00{i}"
    elif i < 100:
        dir_num = f"0{i}"
    else:
        dir_num = f"{i}"
        
    dir_path = [folder for folder in folder_list if dir_num in folder][0]
    file_list = sorted(glob(os.path.join(dir_path, "*.edf")))
    
    full_list = []
    print(f"{dir_path} process...")
    for path in file_list:
        raw = mne.io.read_raw_edf(path, preload = True, verbose = False).pick_types(eeg = True, verbose = False)
        
        if raw.info["sfreq"] != 160:
            continue
        
        sample = raw.get_data()
        info = mne.create_info(ch_list, raw.info["sfreq"], "eeg")
        raw_eeg = mne.io.RawArray(sample, info, verbose = False)
        
        original_channels = raw_eeg.ch_names
        new_order = original_channels.copy()
        new_order = new_ch_list
        
        new_raw = raw_eeg.copy().reorder_channels(new_order)
        raw_array = new_raw.get_data()
        
        segment_time = int(new_raw.info["sfreq"] * 10)
        
        cnt = 0
        accumulate_time = segment_time
        
        segment_list = []
        while accumulate_time <= raw_array.shape[-1]:
            segment = raw_array[:, int(cnt * segment_time):int((cnt+1) * segment_time)]
            segment_list.append(segment[np.newaxis])
            cnt += 1
            accumulate_time += segment_time
            
        segment_concat = np.concatenate(segment_list, axis = 0)
        full_list.append(segment_concat)
    data_concat = np.concatenate(full_list, axis = 0)
    print(data_concat.shape)
        
    dir_list.append(data_concat)

data/eegmmidb/S001 process...
(156, 64, 1600)
data/eegmmidb/S002 process...
(156, 64, 1600)
data/eegmmidb/S003 process...
(156, 64, 1600)
data/eegmmidb/S004 process...
(156, 64, 1600)
data/eegmmidb/S005 process...
(156, 64, 1600)
data/eegmmidb/S006 process...
(156, 64, 1600)
data/eegmmidb/S007 process...
(156, 64, 1600)
data/eegmmidb/S008 process...
(156, 64, 1600)
data/eegmmidb/S009 process...
(156, 64, 1600)
data/eegmmidb/S010 process...
(156, 64, 1600)
data/eegmmidb/S011 process...
(156, 64, 1600)
data/eegmmidb/S012 process...
(156, 64, 1600)
data/eegmmidb/S013 process...
(156, 64, 1600)
data/eegmmidb/S014 process...
(156, 64, 1600)
data/eegmmidb/S015 process...
(156, 64, 1600)
data/eegmmidb/S016 process...
(156, 64, 1600)
data/eegmmidb/S017 process...
(156, 64, 1600)
data/eegmmidb/S018 process...
(156, 64, 1600)
data/eegmmidb/S019 process...
(156, 64, 1600)
data/eegmmidb/S020 process...
(156, 64, 1600)
data/eegmmidb/S021 process...
(156, 64, 1600)
data/eegmmidb/S022 process...
(156

/tmp/ipykernel_2658/4152408956.py:16: RuntimeWarning: Limited 1 annotation(s) that were expanding outside the data range.
  raw = mne.io.read_raw_edf(path, preload = True, verbose = False).pick_types(eeg = True, verbose = False)
/tmp/ipykernel_2658/4152408956.py:16: RuntimeWarning: Limited 1 annotation(s) that were expanding outside the data range.
  raw = mne.io.read_raw_edf(path, preload = True, verbose = False).pick_types(eeg = True, verbose = False)
/tmp/ipykernel_2658/4152408956.py:16: RuntimeWarning: Limited 1 annotation(s) that were expanding outside the data range.
  raw = mne.io.read_raw_edf(path, preload = True, verbose = False).pick_types(eeg = True, verbose = False)
/tmp/ipykernel_2658/4152408956.py:16: RuntimeWarning: Limited 1 annotation(s) that were expanding outside the data range.
  raw = mne.io.read_raw_edf(path, preload = True, verbose = False).pick_types(eeg = True, verbose = False)
/tmp/ipykernel_2658/4152408956.py:16: RuntimeWarning: Limited 1 annotation(s) that w

(12, 64, 1600)
data/eegmmidb/S101 process...
(156, 64, 1600)
data/eegmmidb/S102 process...
(156, 64, 1600)
data/eegmmidb/S103 process...
(156, 64, 1600)
data/eegmmidb/S104 process...
(154, 64, 1600)
data/eegmmidb/S105 process...
(156, 64, 1600)
data/eegmmidb/S106 process...
(147, 64, 1600)
data/eegmmidb/S107 process...
(156, 64, 1600)
data/eegmmidb/S108 process...
(156, 64, 1600)
data/eegmmidb/S109 process...
(156, 64, 1600)


In [12]:
save_data = np.concatenate(dir_list, axis = 0)
save_data.shape

(16567, 64, 1600)

In [23]:
save_data.shape[0] * 0.2

3313.4

In [24]:
13254 + 3313

16567

In [25]:
np.save("data/eegmmidb/eeg_0_80.npy", save_data)

In [2]:
np.load("data/eegmmidb/eeg_0_80.npy").shape

(16567, 64, 1600)